In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
train = pd.read_csv('data/diamonds_train.csv')
test = pd.read_csv('data/diamonds_test.csv')
sample_sub = pd.read_csv('data/sample_submission.csv')

In [66]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id       13485 non-null  int64   
 1   carat    13485 non-null  float64 
 2   cut      13485 non-null  category
 3   color    13485 non-null  category
 4   clarity  13485 non-null  category
 5   depth    13485 non-null  float64 
 6   table    13485 non-null  float64 
 7   x        13485 non-null  float64 
 8   y        13485 non-null  float64 
 9   z        13485 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 778.0 KB


In [40]:
target = 'price'
cat_features = ['cut', 'color', 'clarity']
num_features = ['carat', 'depth', 'table', 'x', 'y', 'z']

for cat_feat in cat_features:
    train[cat_feat] = train[cat_feat].astype('category')
    test[cat_feat] = test[cat_feat].astype('category')
    
cat_df = pd.get_dummies(train[cat_features])
num_df = train.loc[:,num_features]
train_df = pd.concat([cat_df, num_df], axis=1)

cat_df = pd.get_dummies(test[cat_features])
num_df = test.loc[:,num_features]
test_df = pd.concat([cat_df, num_df], axis=1)


features = list(cat_df.columns) + list(num_df.columns)

In [47]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(train_df.loc[:,features].values)
# y = train[target]

X=train_df.loc[:,features]
y=train[target]

In [8]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


In [9]:
# numeric_transformer = \
# Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
#                 ('scaler', StandardScaler())])

In [36]:
# # categorical_transformer = Pipeline(steps=[('nothing',SimpleImputer(strategy='constant', fill_value='missing'))])
# categorical_transformer =Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#                 ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [37]:
# preprocessor = \
# ColumnTransformer(transformers=[('num', numeric_transformer, num_features),
#                                 ('cat', categorical_transformer, cat_features)])

In [38]:
# features=num_features+cat_features
# X=train[features]
# y=train[target]

In [41]:
# X.head()

In [13]:
# from lightgbm import LGBMRegressor

In [42]:
# model = LGBMRegressor()

# model.fit(X, y)

In [15]:
from sklearn.model_selection import cross_val_score

In [16]:
scores = cross_val_score(model, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=10, verbose=2)
np.mean(-scores)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.3s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.6s finished


539.7492636586812

In [17]:
# from sklearn.model_selection import RandomizedSearchCV

In [29]:
# param_grid = {
#     'learning_rate': [0.01, 0.05, 0.1,0.15,0.2, 0.25],
#     'n_estimators': [50,100, 125, 150, 175, 200],
#     'max_depth': [ 4, 8, 16],
#     'num_leaves':[15, 30, 50,100,200]
# }

# grid_search = RandomizedSearchCV(model, 
#                                  param_grid, 
#                                  cv=10, 
#                                  verbose=2, 
#                                  scoring='neg_root_mean_squared_error', 
#                                  n_jobs=-1,
#                                  n_iter=32)

# grid_search.fit(X, y)
# print(grid_search.best_params_)
# print(-grid_search.best_score_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
{'num_leaves': 100, 'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.1}
-540.8149692543371


In [31]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

In [56]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mse', optimizer='adam', metrics=['mse'])
NN_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               3456      
_________________________________________________________________
dense_11 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_12 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_13 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 257       
Total params: 168,321
Trainable params: 168,321
Non-trainable params: 0
_________________________________________________________________


In [57]:
checkpoint_name = '/home/alvaro/NN_model2/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [58]:
NN_model.fit(X, y, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
1012/1012 [==============================] - 3s 3ms/step - loss: 13356787.8672 - mse: 13356787.8672 - val_loss: 965672.5000 - val_mse: 965672.5000

Epoch 00001: val_loss improved from inf to 965672.50000, saving model to /home/alvaro/NN_model2/Weights-001--965672.50000.hdf5
Epoch 2/500
1012/1012 [==============================] - 3s 3ms/step - loss: 1105601.1987 - mse: 1105601.1987 - val_loss: 609841.1875 - val_mse: 609841.1875

Epoch 00002: val_loss improved from 965672.50000 to 609841.18750, saving model to /home/alvaro/NN_model2/Weights-002--609841.18750.hdf5
Epoch 3/500
1012/1012 [==============================] - 3s 3ms/step - loss: 980237.9350 - mse: 980237.9350 - val_loss: 720540.0625 - val_mse: 720540.0625

Epoch 00003: val_loss did not improve from 609841.18750
Epoch 4/500
1012/1012 [==============================] - 3s 3ms/step - loss: 659610.8295 - mse: 659610.8295 - val_loss: 519269.7500 - val_mse: 519269.7500

Epoch 00004: val_loss improved from 609841.18750 to

In [59]:
weights_file = '/home/alvaro/NN_model2/Weights-474--279609.09375.hdf5' # choose the best checkpoint 
NN_model.load_weights(weights_file) # load it
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [60]:
predictions = NN_model.predict(X)
from sklearn.metrics import mean_squared_error
print(f"train error: {mean_squared_error(y_pred=predictions, y_true=y, squared=False)}")

train error: 507.37809222042387


In [74]:
y_pred = NN_model.predict(test_df.loc[:,features])
y_pred=[x[0] for x in y_pred]
submission_df = pd.DataFrame({'id': test['id'], 'price': y_pred})
submission_df.head()
submission_df.to_csv('submission_NN.csv', index=False)

In [26]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', GradientBoostingRegressor())])
model.fit(X=X, y=y)
print("model created!")

model created!


In [24]:
y_train = model.predict(X)

# Metricas

In [11]:
from sklearn.metrics import mean_squared_error

In [27]:
print(f"train error: {mean_squared_error(y_pred=y_train, y_true=y, squared=False)}")

train error: 207.0260781966406


# Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(model, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=10, verbose=2)
np.mean(-scores)

# Grid search

In [29]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean'],
    'regressor__learning_rate': [0.2],
    'regressor__learning_rate': [0.15,0.2, 0.25, 0.3],
    'regressor__n_estimators': [100,125, 150, 175, 200],
    'regressor__max_depth': [4, 8, 16],
    'regressor__subsample': [1] ,
    'regressor__min_samples_leaf': [1] 
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=2, 
                                 scoring='neg_root_mean_squared_error', 
                                 #n_jobs=-1,
                                 n_iter=32)

grid_search.fit(X, y)

In [43]:
grid_search.best_params_

{'regressor__subsample': 1,
 'regressor__n_estimators': 125,
 'regressor__min_samples_leaf': 1,
 'regressor__max_depth': 8,
 'regressor__learning_rate': 0.15,
 'preprocessor__num__imputer__strategy': 'mean'}

In [44]:
-grid_search.best_score_

549.977089446955

# **Submission**

In [45]:
y_pred = grid_search.predict(test[features])
submission_df = pd.DataFrame({'id': test['id'], 'price': y_pred})
submission_df.head()
submission_df.to_csv('submission_GBM_grid.csv', index=False)